In [1]:
import numpy as np
import torch
import os
import shutil

In [2]:
data_root = 'DJI ROCO'

In [3]:
os.listdir(data_root)

['readme.txt',
 'robomaster_South China Regional Competition',
 'robomaster_Final Tournament',
 'robomaster_Central China Regional Competition',
 'Copyright Statement.doc',
 'robomaster_North China Regional Competition']

In [4]:
test_image_root = os.path.join(data_root, 'robomaster_Central China Regional Competition', 'image')

In [5]:
test_image_paths = [os.path.join(test_image_root, i) for i in os.listdir(test_image_root)]

In [6]:
test_image_paths[0].replace('image', 'image_annotation').replace('jpg', 'xml')

'DJI ROCO/robomaster_Central China Regional Competition/image_annotation/电创VsRPS_BO2_2_46.xml'

In [16]:
dst_image_root_path = '/home/cell/hdh/rm/DJI ROCO/train/images'
dst_xml_root_path = '/home/cell/hdh/rm/DJI ROCO/train/xmls'

In [17]:
image_id = 0

In [25]:
# image_root_path = os.path.join(data_root, 'robomaster_Central China Regional Competition', 'image')
# image_root_path = os.path.join(data_root, 'robomaster_North China Regional Competition', 'image')
image_root_path = os.path.join(data_root, 'robomaster_Final Tournament', 'image')

In [26]:
image_paths = [os.path.join(image_root_path, i) for i in os.listdir(image_root_path)]

In [27]:
for i, image_path in enumerate(image_paths):
    xml_path = image_path.replace('image', 'image_annotation').replace('jpg', 'xml')
    if not os.path.exists(xml_path):
        print('error read xml: ', xml_path)
        break
    dst_image_path = os.path.join(dst_image_root_path, f'{image_id}.jpg')
    dst_xml_path = os.path.join(dst_xml_root_path, f'{image_id}.xml')
#     print(image_path, "   to   ", dst_image_path)
#     print(xml_path, "   to   ", dst_xml_path)
    shutil.copy(image_path, dst_image_path)
    shutil.copy(xml_path, dst_xml_path)
    image_id += 1

In [8]:
print(len(os.listdir(os.path.join(data_root, 'robomaster_Central China Regional Competition', 'image'))))
print(len(os.listdir(os.path.join(data_root, 'robomaster_North China Regional Competition', 'image'))))
print(len(os.listdir(os.path.join(data_root, 'robomaster_Final Tournament',  'image'))))

2655
387
2685


In [39]:
print(len(os.listdir('/home/cell/hdh/rm/DJI ROCO/train/images')))

5727


## 处理得到txt

In [2]:
import xml.dom.minidom as minidom

In [3]:
img_root_path = '/home/cell/hdh/yolo/data/rm_data/train/images'
xml_root_path = '/home/cell/hdh/yolo/data/rm_data/train/xmls'
image_paths = [os.path.join(img_root_path, i) for i in os.listdir(img_root_path)]
xml_paths = [os.path.join(xml_root_path, i) for i in os.listdir(xml_root_path)]

分类方法：
- car
- watcher
- base
- ignore
- armor

In [8]:
image_paths

['/home/cell/hdh/yolo/data/rm_data/train/images/5519.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/2699.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/2571.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/4591.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/2672.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/4789.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/1871.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/1675.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/323.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/196.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/202.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/482.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/1137.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/4681.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/3294.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/3956.jpg',
 '/home/cell/hdh/yolo/data/rm_data/train/images/364.jpg',
 '

In [4]:
def get_boxes(xml_path):
    with open(xml_path, 'r') as f:
        dom = minidom.parse(f)
        root = dom.documentElement
        objects = root.getElementsByTagName('object')
        boxes = []
        names = []
        id_dict = {'car':0, 'watcher':1, 'base':2, 'ignore':3, 'armor':4}
        for object_ in objects:
            name = object_.getElementsByTagName('name')[0].firstChild.data
            box = object_.getElementsByTagName('bndbox')[0]

            x1 = box.getElementsByTagName('xmin')[0].firstChild.data
            y1 = box.getElementsByTagName('ymin')[0].firstChild.data
            x2 = box.getElementsByTagName('xmax')[0].firstChild.data
            y2 = box.getElementsByTagName('ymax')[0].firstChild.data
            x1, y1, x2, y2 = map(float, [x1, y1, x2, y2])

            boxes.append([id_dict[name], x1, y1, x2, y2])
            names.append(name)
        return boxes, names

#### 先写一个tran.txt给yolov4测试看看

In [85]:
shutil.rmtree('/home/cell/hdh/rm/data/train/xmls/.ipynb_checkpoints')

In [100]:
lines = []
for image_path in image_paths:
    line_str = ''
    xml_path = image_path.replace('images', 'xmls').replace('jpg', 'xml')
    boxes, names = get_boxes(xml_path)
    if not os.path.exists(image_path.replace('images', 'xmls').replace('jpg', 'xml')):
        print('error: ',image_path.replace('images', 'xmls').replace('jpg', 'xml'))
        break
    line_str += image_path
    line_str += ' '
    for box in boxes:
        line_str += (str(box[1])+','+str(box[2])+','+str(box[3])+','+str(box[4])+','+str(box[0])+' ')
    lines.append(line_str)

In [104]:
with open('train.txt', 'w') as f:
    for line_str in lines:
        f.write(line_str)

#### 写给yolov3

In [13]:
dst_save_path = '/home/cell/hdh/yolo/data/rm_data/train/labels'
# os.mkdir(dst_save_path)

In [17]:
for xml_path in xml_paths:
    dst_save_path = xml_path.replace('xmls', 'labels').replace('xml', 'txt')
    print(dst_save_path)
    boxes, names = get_boxes(xml_path)
    boxes = np.array(boxes)
    if len(boxes) == 0:
        np.savetxt(dst_save_path, np.array([]))
        continue
    boxes[:, 3] = boxes[:, 3] - boxes[:, 1]
    boxes[:, 4] = boxes[:, 4] - boxes[:, 2]
    boxes[:, 1] += (boxes[:, 3] / 2)
    boxes[:, 2] += (boxes[:, 4] / 2)
    np.savetxt(dst_save_path, boxes)

/home/cell/hdh/yolo/data/rm_data/train/labels/2246.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/1605.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/3104.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/883.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/458.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/4776.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/5434.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/3413.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/2063.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/450.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/2149.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/3103.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/2966.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/4867.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/2788.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/2291.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/966.txt
/home/cell/hdh/yolo/data/rm_data/train/labels/2775.txt
/home/cell/hdh

In [18]:
len(os.listdir('/home/cell/hdh/yolo/data/rm_data/train/labels'))

5727

写train.csv和valid.csv

In [5]:
train_csv_path = '/home/cell/hdh/yolo/data/rm_data/train/train.csv'
valid_csv_path = '/home/cell/hdh/yolo/data/rm_data/train/valid.csv'

In [6]:
from utils.utils import *
from utils.preprocess import *

In [7]:
ratio = 0.9
train_image_label = {k:k.replace('images', 'labels').replace('jpg', 'txt') for k in image_paths[:int(len(image_paths)*ratio)]}
valid_image_label = {k:k.replace('images', 'labels').replace('jpg', 'txt') for k in image_paths[int(len(image_paths)*ratio):]}

In [10]:
write_csv(train_image_label, train_csv_path)
write_csv(valid_image_label, valid_csv_path)

In [11]:
import xml.dom.minidom as minidom

test_xml_path = "/home/cell/hdh/yolov5/data/RM_data/DJI ROCO/robomaster_Final Tournament/image_annotation/VW_CH3风暴Vs华南虎_BO2_1_100.xml"
f = open(test_xml_path, 'r')
dom = minidom.parse(f)
root = dom.documentElement
objects = root.getElementsByTagName('object')

In [13]:
test_object = objects[0]

In [28]:
t = test_object.childNodes[0].nodeName

In [35]:
t = test_object.childNodes[3]

In [36]:
t.attributes

1